In [1]:
import pandas as pd


In [4]:
dataset = pd.read_csv("Social_Network_Ads.csv")
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [5]:
dataset = dataset.drop("User ID", axis=1)

In [6]:
dataset = pd.get_dummies(dataset, dtype=int, drop_first=True)
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [7]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [9]:
independent = dataset[['Age', 'EstimatedSalary',  'Gender_Male']]
independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [10]:
dependent = dataset[['Purchased']]
dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [11]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(independent, dependent, random_state=0, test_size=0.30)

In [12]:
from sklearn.preprocessing import StandardScaler
scx = StandardScaler()
x_train = scx.fit_transform(x_train)
x_test = scx.transform(x_test)

In [26]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
    "criterion":["gine", "entrophy", "log_loss"],
    "max_features": ["sqrt", "log2", None],
    "splitter": ["best", "random"]
}

grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit=True, verbose=3, cv=5, n_jobs=-1, scoring="f1_weighted")

grid.fit(x_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
60 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
24 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Anaconda3\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\Anaconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\Anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py", line 95, in validate_pa

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gine', 'entrophy', 'log_loss'],
                         'max_features': ['sqrt', 'log2', None],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [27]:
y_pred = grid.predict(x_test)

In [28]:
from sklearn.metrics import f1_score
f1_macro = f1_score(y_test, y_pred, average="weighted")
print("f1 macro value for the best parameter is {}".format(grid.best_params_), f1_macro)

f1 macro value for the best parameter is {'criterion': 'log_loss', 'max_features': 'sqrt', 'splitter': 'best'} 0.8919742664927225


In [29]:
from sklearn.metrics import roc_auc_score
roc_score = roc_auc_score(y_test, grid.predict_proba(x_test)[:,1])
print("roc_auc score for the model is {}:".format(roc_score))

roc_auc score for the model is 0.8825254708243284:


In [32]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
cm = confusion_matrix(y_test, y_pred)
clf_report=classification_report(y_test, y_pred)
print(clf_report)

print(cm)


              precision    recall  f1-score   support

           0       0.92      0.91      0.92        79
           1       0.83      0.85      0.84        41

    accuracy                           0.89       120
   macro avg       0.88      0.88      0.88       120
weighted avg       0.89      0.89      0.89       120

[[72  7]
 [ 6 35]]


In [34]:
re = grid.cv_results_
table = pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001599,0.003198,0.000000,0.000000,gine,sqrt,best,"{'criterion': 'gine', 'max_features': 'sqrt', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
1,0.000000,0.000000,0.000000,0.000000,gine,sqrt,random,"{'criterion': 'gine', 'max_features': 'sqrt', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
2,0.003200,0.003919,0.000000,0.000000,gine,log2,best,"{'criterion': 'gine', 'max_features': 'log2', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
3,0.000000,0.000000,0.000000,0.000000,gine,log2,random,"{'criterion': 'gine', 'max_features': 'log2', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
4,0.001600,0.003200,0.000000,0.000000,gine,None,best,"{'criterion': 'gine', 'max_features': None, 's...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
5,0.000000,0.000000,0.000000,0.000000,gine,None,random,"{'criterion': 'gine', 'max_features': None, 's...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
6,0.000000,0.000000,0.000000,0.000000,entrophy,sqrt,best,"{'criterion': 'entrophy', 'max_features': 'sqr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
7,0.001601,0.003202,0.000000,0.000000,entrophy,sqrt,random,"{'criterion': 'entrophy', 'max_features': 'sqr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
8,0.000000,0.000000,0.000000,0.000000,entrophy,log2,best,"{'criterion': 'entrophy', 'max_features': 'log...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
9,0.001609,0.003218,0.000000,0.000000,entrophy,log2,random,"{'criterion': 'entrophy', 'max_features': 'log...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
